In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [3]:
from pyspark.ml.clustering import KMeans

In [4]:
dataset = spark.read.format('libsvm').load('FileStore/tables/sample_kmeans_data.txt')

In [5]:
dataset.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0| (3,[],[])|
 1.0|(3,[0,1,2],[0.1,0...|
 2.0|(3,[0,1,2],[0.2,0...|
 3.0|(3,[0,1,2],[9.0,9...|
 4.0|(3,[0,1,2],[9.1,9...|
 5.0|(3,[0,1,2],[9.2,9...|
+-----+--------------------+

In [6]:
final_data = dataset.select('features')

In [7]:
final_data.show()

+--------------------+
 features|
+--------------------+
 (3,[],[])|
(3,[0,1,2],[0.1,0...|
(3,[0,1,2],[0.2,0...|
(3,[0,1,2],[9.0,9...|
(3,[0,1,2],[9.1,9...|
(3,[0,1,2],[9.2,9...|
+--------------------+

In [8]:
kmeans = KMeans().setK(2).setSeed(1)

In [9]:
model = kmeans.fit(final_data) 
# kmeans only expects feature column(no label column because it is unsupervised learning)

In [10]:
# wssse: within set sum of squared errors
wssse = model.computeCost(final_data)

In [11]:
print(wssse)

0.11999999999994547

In [12]:
# check out the centroids of the clusters found by the model
centers = model.clusterCenters()

In [13]:
centers

Out[14]: [array([0.1, 0.1, 0.1]), array([9.1, 9.1, 9.1])]

In [14]:
# see the labeled cluster
results = model.transform(final_data)

In [15]:
results.show()

+--------------------+----------+
 features|prediction|
+--------------------+----------+
 (3,[],[])| 0|
(3,[0,1,2],[0.1,0...| 0|
(3,[0,1,2],[0.2,0...| 0|
(3,[0,1,2],[9.0,9...| 1|
(3,[0,1,2],[9.1,9...| 1|
(3,[0,1,2],[9.2,9...| 1|
+--------------------+----------+

In [16]:
# KMeans with a more realistic problem
# classify seed types
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [17]:
dataset = spark.read.csv('FileStore/tables/seeds_dataset.csv', header = True, inferSchema = True)

In [18]:
dataset.printSchema()

root
-- area: double (nullable = true)
-- perimeter: double (nullable = true)
-- compactness: double (nullable = true)
-- length_of_kernel: double (nullable = true)
-- width_of_kernel: double (nullable = true)
-- asymmetry_coefficient: double (nullable = true)
-- length_of_groove: double (nullable = true)

In [19]:
dataset.head(1)
# features are all numerical

Out[20]: [Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)]

In [20]:
from pyspark.ml.feature import VectorAssembler

In [21]:
dataset.columns

Out[22]: ['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [22]:
assembler = VectorAssembler(inputCols = dataset.columns, outputCol = 'features')

In [23]:
final_data = assembler.transform(dataset)

In [24]:
final_data.printSchema()

root
-- area: double (nullable = true)
-- perimeter: double (nullable = true)
-- compactness: double (nullable = true)
-- length_of_kernel: double (nullable = true)
-- width_of_kernel: double (nullable = true)
-- asymmetry_coefficient: double (nullable = true)
-- length_of_groove: double (nullable = true)
-- features: vector (nullable = true)

In [25]:
from pyspark.ml.feature import StandardScaler

In [26]:
scaler = StandardScaler(inputCol = 'features', outputCol = 'scaledFeatures')

In [27]:
scaler_model = scaler.fit(final_data)

In [28]:
final_data = scaler_model.transform(final_data)

In [29]:
final_data.head(1)

Out[33]: [Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))]

In [30]:
kmeans = KMeans(featuresCol = 'scaledFeatures', k = 3)

In [31]:
model = kmeans.fit(final_data)

In [32]:
print('WSSSE')
print(model.computeCost(final_data))

WSSSE
429.07559671506715

In [33]:
centers = model.clusterCenters()
centers

Out[39]: [array([ 4.87257659, 10.88120146, 37.27692543, 12.3410157 , 8.55443412,
 1.81649011, 10.32998598]),
 array([ 4.06105916, 10.13979506, 35.80536984, 11.82133095, 7.50395937,
 3.27184732, 10.42126018]),
 array([ 6.31670546, 12.37109759, 37.39491396, 13.91155062, 9.748067 ,
 2.39849968, 12.2661748 ])]

In [34]:
model.transform(final_data).select('prediction').show()

+----------+
prediction|
+----------+
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 2|
 2|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 1|
+----------+
only showing top 20 rows